In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"qasmovdyant","key":"4d4759066dca189587bcac8981b34740"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sehriyarmemmedli/binaaz-sale-project

Dataset URL: https://www.kaggle.com/datasets/sehriyarmemmedli/binaaz-sale-project
License(s): unknown
 93% 37.0M/39.8M [00:00<00:00, 90.9MB/s]
100% 39.8M/39.8M [00:00<00:00, 90.2MB/s]


In [ ]:
!unzip /content/binaaz-sale-project.zip

Archive:  /content/binaaz-sale-project.zip
  inflating: house_sale.csv          


In [ ]:
df = pd.read_csv('/content/house_sale.csv')

In [ ]:
df1=df.copy()

In [ ]:
df.drop(['id_x', 'price', 'id_y', 'currency_x', 'estate_details_id_y', 'estate_id', 'estate_rel_url_y',
         'estate_details_id_x', 'datetime_scrape_x', 'datetime_scrape_y', 'day_x', 'hour_x', 'day_y',
         'hour_y', 'updated', 'lat', 'lng', 'address', 'owner_name', 'owner_title', 'shop_name', 'shop_title',
         'img_url', 'description','bill_of_sale', 'views', 'attributes', 'Binanın növü', 'city_when',
         'unit_price', 'mortgage', 'vip', 'repair', 'featured', 'products_label', 'currency_y','rel_url', 'estate_rel_url_x',
         'estate_rel_url', 'extra_info'], axis=1, inplace = True)

In [ ]:
df.columns

Index(['location', 'city', 'total_price', 'Kateqoriya', 'Mərtəbə', 'Otaq sayı',
       'Sahə', 'Torpaq sahəsi', 'Təmir', 'Çıxarış', 'İpoteka'],
      dtype='object')

In [ ]:
df

,location,city,total_price,Kateqoriya,Mərtəbə,Otaq sayı,Sahə,Torpaq sahəsi,Təmir,Çıxarış,İpoteka
0,Səbail r.,bakı,499999.0,Köhnə tikili,7 / 9,4.0,145 m²,NaN,var,var,NaN
1,Biləcəri q.,bakı,77000.0,Həyət evi/Bağ evi,NaN,4.0,90 m²,1.3 sot,var,yoxdur,NaN
2,İnşaatçılar m.,bakı,92000.0,Həyət evi/Bağ evi,NaN,3.0,60 m²,0.1 sot,var,var,NaN
3,Qaraçuxur q.,bakı,95000.0,Obyekt,NaN,NaN,130 m²,NaN,var,var,var
4,Əhmədli m.,bakı,220000.0,Yeni tikili,15 / 16,3.0,100 m²,NaN,var,var,NaN
...,...,...,...,...,...,...,...,...,...,...,...
100770,Binəqədi q.,bakı,170000.0,Həyət evi/Bağ evi,NaN,6.0,250 m²,3.5 sot,var,yoxdur,NaN
100771,Mərdəkan q.,bakı,520000.0,Həyət evi/Bağ evi,NaN,4.0,220 m²,11 sot,var,var,var
100772,Mərdəkan q.,bakı,650000.0,Həyət evi/Bağ evi,NaN,5.0,350 m²,7 sot,var,var,var
100773,20 Yanvar m.,bakı,310000.0,Yeni tikili,4 / 19,3.0,130 m²,NaN,var,var,var


In [ ]:
df['Təmir'].value_counts(dropna=False)

,count
Təmir,
var,81612
yoxdur,13162
NaN,6001


In [ ]:
df.isna().sum()

,0
location,0
city,0
total_price,0
Kateqoriya,0
Mərtəbə,24779
Otaq sayı,9412
Sahə,0
Torpaq sahəsi,85204
Təmir,6001
Çıxarış,0


In [ ]:
#NaN olan dəyərləri 0, "Ipoteka" olanları isə 1 ilə əvəz et
df['İpoteka'] = df['İpoteka'].notna().astype(int)

In [ ]:
df[['Torpaq sahəsi']] = df[['Torpaq sahəsi']].fillna("0")

In [ ]:
df[['Mərtəbə']] = df[['Mərtəbə']].fillna("0")

In [ ]:
# df[['Otaq sayı']] = df[['Otaq sayı']].fillna("0")
df['Otaq sayı'] = df['Otaq sayı'].fillna(df['Otaq sayı'].mean())

In [ ]:
df['Təmir'] = df['Təmir'].fillna(df['Təmir'].mode()[0])

In [ ]:
df['Sahə'] = df['Sahə'].str.replace(r'sot', '', regex=True)
df['Sahə'] = df['Sahə'].str.replace(' ', '', regex=True) # Remove spaces

In [ ]:
# Torpaq sahəsi sütununda 'sot' sözünü təmizləyirik
df['Torpaq sahəsi'] = df['Torpaq sahəsi'].str.replace(r' sot', '', regex=True)
df['Torpaq sahəsi'] = df['Torpaq sahəsi'].str.replace(' ', '', regex=True) # Remove spaces

In [ ]:
# Sahə sütununda 'm²' sözünü təmizləyirik
df['Sahə'] = df['Sahə'].str.replace(r'm²', '', regex=True)
df['Sahə'] = df['Sahə'].str.replace(' ', '', regex=True) # Remove spaces

In [ ]:
# LabelEncoder instansiyasını yaradıb bezi sütunlari kodlaşdırırıq
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['location'] = encoder.fit_transform(df['location'])
df['city'] = encoder.fit_transform(df['city'])
df['Kateqoriya'] = encoder.fit_transform(df['Kateqoriya'])
df['Mərtəbə'] = encoder.fit_transform(df['Mərtəbə'])
df['Otaq sayı'] = encoder.fit_transform(df['Otaq sayı'])
df['Mərtəbə'] = encoder.fit_transform(df['Mərtəbə'])
df['Sahə'] = encoder.fit_transform(df['Sahə'])
df['Torpaq sahəsi'] = encoder.fit_transform(df['Torpaq sahəsi'])
df['Təmir'] = encoder.fit_transform(df['Təmir'])
df['Çıxarış'] = encoder.fit_transform(df['Çıxarış'])

In [ ]:
X = df.drop('Çıxarış', axis = 1)
y = df['Çıxarış'].copy()

In [ ]:
num_features = ['total_price', 'Otaq sayı', 'Sahə', 'Torpaq sahəsi']
cat_features = ['location', 'city', 'İpoteka', 'Kateqoriya', 'Mərtəbə']

# Boş dəyərləri dolduran və miqyaslayan pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Kateqorik dəyərləri kodlaşdıran pipeline
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Bütün pre-processing addımlarını birləşdirən pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])


In [ ]:
# Train və Test setlərini böl
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Model qur
model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced'))
])

In [ ]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['total_price', 'Otaq sayı',
                                                   'Sahə', 'Torpaq sahəsi']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['location', 'city',
                                                   'İpoteka', 'Kateqoriya',
                                                   'Mərtəbə'])])),
                ('classifier',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=200, random_state=42))])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = model.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nAccuracy:", accuracy_score(y_test, y_pred))

Confusion Matrix:
 [[15333   580]
 [  693  3549]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96     15913
           1       0.86      0.84      0.85      4242

    accuracy                           0.94     20155
   macro avg       0.91      0.90      0.90     20155
weighted avg       0.94      0.94      0.94     20155


Accuracy: 0.9368394939221037


In [ ]:
rfr.fit(X_train, y_train)

NameError: name 'rfr' is not defined

In [ ]:
model.score(X_train,y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
#max_depth və min_samples_leaf ilə incə tənzimləmə
rf = RandomForestClassifier(n_estimators=200, max_depth=15, min_samples_leaf=5, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
#Feature Selection (Ən önəmli xüsusiyyətləri istifade et:)
feature_importances = rf.feature_importances_
feature_names = X_train.columns

# Ən vacib xüsusiyyətləri çap edirik
important_features = pd.Series(feature_importances, index=feature_names).sort_values(ascending=False)
print(important_features.head(10))  # Ən önəmli 10 xüsusiyyəti göstərir


In [ ]:
#Cross-Validation istifadə edirik (Test nəticəsini stabilləşdirmək üçün)
from sklearn.model_selection import cross_val_score

rf = RandomForestClassifier(n_estimators=200, max_depth=15, min_samples_leaf=5, random_state=42)
scores = cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Score: {scores.mean():.2f}")

In [ ]:
#Daha çox ağac (n_estimators) əlavə etmək lazımdırsa yoxla:
rf = RandomForestClassifier(n_estimators=500, max_depth=15, min_samples_leaf=5, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_train,y_train)

In [ ]:
rf.score(X_test, y_test)